In [ ]:
import sys
from CoRe_Dataloader_ECSG import *
from CoRe_Dataloader_From_File_With_Random_From_Tensors import *
import torch
import numpy as np
import matplotlib.pyplot as plt
import plotly as pt 
import plotly.graph_objects as go 
from  watpy.coredb.coredb import *
cdb = CoRe_db("./CoRe_DB/")


In [ ]:
a = CoRe_DB_Dataset()

In [ ]:
a

In [ ]:
# create a function to create a layout for plotly after taking in the x axes title, y axes title, and plot title 
def get_layout(xtitle: str, ytitle: str, title: str, height: int, width: int) -> go.Layout:
    primary_color = rgba(133, 162, 204)
    secondary_color = rgba(168, 217, 255)
    grid_color = rgba(83, 107, 126)
    return go.Layout(
        title=dict(text=title,
                   font=dict(size=90, family="Verdana",color=primary_color),
                   y = .98,
        ),
        xaxis=dict(title=dict(text=xtitle, font=dict(size=60, family="Verdana", color=secondary_color)),
                    showline=False,
                    linewidth=2,
                    linecolor=grid_color,
                    mirror=True,
                    showgrid=True,
                    gridwidth=2,
                    gridcolor=grid_color,
                    ),
        yaxis=dict(title=dict(text=ytitle, font=dict(size=60, family="Verdana", color=secondary_color)),
                    showline=False,
                    linewidth=2,
                    linecolor=grid_color,
                    mirror=True,
                    showgrid=True,
                    gridwidth=2,
                    gridcolor=grid_color,
                    ),
        showlegend=True,
        legend=dict(
                    font=dict(size=60, family="Verdana",color=secondary_color),
                    itemwidth = 90,
                    itemsizing = "constant",
                    orientation = "h",
                    xanchor = "center",
                    yanchor = "top",
                    x = .5,
                    y = 1
                    ),
        margin = dict(t = 200),
        font=dict(size=25, family="Verdana", color=secondary_color),
        width=width,
        height=height,
        plot_bgcolor=rgba(0,0,0,255),
        paper_bgcolor=rgba(0,0,0,255),
    )
def pplt(data: List[object], xtitle: str = "XTITLE the quick brown fox jumped over the lazy dog", ytitle: str = "YTITLE the quick brown fox jumped over the lazy dog", title: str = "PLOTTITLE the quick brown fox jumped over the lazy dog", height: int = -1, width: int = -1):
    fig = go.Figure(data=data, layout=get_layout(xtitle, ytitle, title, height, width))
    return fig
def rgba(r:int = 0,g:int = 0,b:int = 0,a:int = 255) -> str:
    return f"rgba({r},{g},{b},{a})"

In [ ]:
example_total_data = cdb.sim["THC:0087"].run["R01"].data.read("rh_22").T

In [ ]:
initial_time = example_total_data[0]*4.975e-6
initial_rstr = example_total_data[1]
initial_istr = example_total_data[2]
initial_ener = (example_total_data[1]**2 + example_total_data[2]**2)

In [ ]:
postmerger_rstr = cut_at_lowest_envelope(initial_rstr, initial_istr)
postmerger_time = initial_time[-len(postmerger_rstr):]
postmerger_rstr.shape, postmerger_time.shape


In [ ]:
lts = len(postmerger_time)

In [ ]:
window = planck_window(math.floor(math.log(lts/2)*6), lts+2)
windowx = np.linspace(0, len(window), len(window))

In [ ]:
post_planck_rstr = np.multiply(postmerger_rstr, window,)

In [ ]:
plt.plot(post_planck_rstr)

In [ ]:
postmerger_time

In [ ]:
from scipy import signal
morlet = signal.morlet(1000,w=10,s = .6)

In [ ]:
sam_p = (postmerger_time[-1] - postmerger_time[0])/len(postmerger_time)


In [ ]:
# postmerger_awt = pad_width(wt(postmerger_rstr, sam_p))


In [ ]:
# plt.pcolormesh(postmerger_awt)

In [ ]:
# noised = np.random.normal(loc = 0,scale = 0.001/3.5, size = (400,400)) + postmerger_awt

In [ ]:
fig = pplt(
    [
        # go.Heatmap(z=noised, colorscale="Ice",
        #            colorbar=dict(title='Energy')),

        go.Scatter(x=initial_time, y=initial_rstr, name="Complete Merger Waveform", line=dict(
            width=4, color=rgba(255, 255, 255))),
        # go.Scatter(x=postmerger_time, y=postmerger_rstr, name="Postmerger Only Waveform", line=dict(
        #     width=4, color=rgba(140, 255, 205))),
        go.Scatter(x=initial_time, y=initial_ener, name="Total Waveform Energy", line=dict(
            width=4, color=rgba(255, 200-20, 200-20))),

    ],
    height=1000,
    width=3000, title="Complete Waveform Strain and Energy Vs. Time [THC:0087]", ytitle="Strain", xtitle="Time [ms]")
fig.update_yaxes(range=[-.25, .25])
fig.show()


In [ ]:
import pandas as pd

In [ ]:
# idf = pd.read_csv(r"C:\Users\aashr\Desktop\research\testing_grounds\total_corrects_and_factors.pd.csv")
# ieosdf = pd.read_csv(r"C:\Users\aashr\Desktop\research\testing_grounds\accuracy_by_eos.pd.csv")
# isnrdf = pd.read_csv(r"C:\Users\aashr\Desktop\research\testing_grounds\accuracy_by_signal-to-noise-ratio.pd.csv")

# idf = idf.drop("Unnamed: 0",axis = 1)
# ieosdf = ieosdf.drop("Unnamed: 0",axis = 1)
# isnrdf = isnrdf.drop("Unnamed: 0",axis = 1)
# eosdf = ieosdf.T.reset_index().rename(
#     columns={"index": "EOS", 0: "accuracy"})


# eosdf["accuracy"] = eosdf["accuracy"]*100
# eosdf

# fig = pplt(
#     [
#         go.Bar(x = eosdf["EOS"],y = eosdf["accuracy"], marker_color=rgba(255, 255, 255),name = "Accuracy by EOS"),

#     ],
#     height=2500,
#     width=2000, title="Accuracy by Equation of State", ytitle="Accuracy [90% to 100%]", xtitle="Equation of State [19 Total]")
# fig.update_yaxes(range=[90, 100])
# fig.update_layout(showlegend = False,font = dict(size = 40))
# fig.show()

# fig = pplt(
#     [
#         go.Bar(x=eosdf["EOS"], y=eosdf["accuracy"], marker_color=rgba(
#             255, 255, 255), name="Accuracy by EOS"),

#     ],
#     height=2500,
#     width=2000, title="Accuracy by Equation of State", ytitle="Accuracy [90% to 100%]", xtitle="Time [ms]")
# fig.update_yaxes(range=[90, 100])
# fig.update_layout(showlegend=False, font=dict(size=40))
# fig.show()

# snrdf = isnrdf
# snrdf = snrdf.T.reset_index().rename(
#     columns={"index": "SNR", 0: "accuracy"})
# snrdf["SNR"] = snrdf["SNR"].astype(float).round(decimals=4).astype(str)

# snrdf["SNR"][0] = "No Noise"
# snrdf
# fig = pplt(
#     [
#         go.Bar(y=snrdf["SNR"], x=snrdf["accuracy"]*100, marker_color=rgba(
#             255, 255, 255), name="Accuracy by EOS",orientation = "h"),

#     ],
#     height=2500,
#     width=2000, title="Accuracy by Signal to Noise Ratio", ytitle="Signal to Noise Ratio", xtitle="Accuracy [40% to 100%]")
# fig.update_xaxes(range=[40, 100])
# fig.update_layout(showlegend=False, font=dict(size=40))
# fig.show()


# remapper = {value: key for key, value in dataset.eosmap.items()}
# df["EOS"].replace(remapper, inplace=True)
# eoscounts = pd.DataFrame(df["EOS"])

# eoscounts
# eoscounts.count()
# a = eoscounts["EOS"].value_counts()
# eoscounts = pd.DataFrame(a).reset_index().rename(columns={"index": "EOS", "EOS": "Occurences"})
# # eoscounts["Occurences"] /= eoscounts["Occurences"].sum()
# # eoscounts = eoscounts.rename(columns={"Occurences":"pm"})
# eoscounts
# fig = pplt(
#     [
#         go.Bar(x=eoscounts["EOS"], y=eoscounts["pm"]*100, marker_color=rgba(
#             255, 255, 255), name="Accuracy by EOS"),

#     ],
#     height=1000,
#     width=3000, title="Percentage Makeup of Dataset by Equation of State", ytitle="Percentage of Dataset", xtitle="Equation of State")
# fig.update_layout(showlegend=False, font=dict(size=40),margin = dict(l = 150))
# fig.show()

# a = torch.load("./roc.pt")
# fpr,tpr,thresholds  = a
# fpr
# eosplots = []
# for i,j,k in zip(fpr,tpr,remapper.items()):
#     eosplots.append(go.Scatter(x=list(i), y=(j), name=k[1], line=dict(
#         width=8,)))
#     print(k[1])

# fig = pplt(eosplots,
#     height=2000,
#     width=5200, title="ROC Curves per Class (EOS)", ytitle="True Positive Rate", xtitle="False Positive Rate")
# fig.update_layout(font=dict(size=80), margin=dict(l=150, t=150), legend=dict(
#     font=dict(size=80),
#     itemwidth=60,
#     itemsizing="constant",
#     orientation="v",
#     xanchor="center",
#     yanchor="top",
#     x=1.1,
#     y=1
# ), title=dict(
#               font=dict(size=200),
#               y=.98,
#               ),)
# fig.show()


In [ ]:
# import vit
# import vit_pytorch
from vit_pytorch.extractor import Extractor
# from vit_pytorch.recorder import Recorder
from vit_pytorch import vit_for_small_dataset as vit_sd
from vit_pytorch import vit as simple_vit
from vit_pytorch.deepvit import DeepViT


def init_model():
    # return simple_vit.ViT(image_size=400,
    #                patch_size=20,
    #                num_classes=19,
    #                dim=int(1024/2),
    #                depth=2,
    #                heads=8,
    #                mlp_dim=int(2048/2),
    #                channels=1).to("cuda:0")
    # return vit_sd.ViT(image_size=400,
    #                patch_size=20,
    #                num_classes=19,
    #                dim=1024,
    #                depth=4,
    #                heads=16,
    #                mlp_dim=int(2048/2),
    #                dropout = 0.1,
    #                emb_dropout = 0,
    #                channels=1).to("cuda:0")
    return DeepViT(image_size=400,
                   patch_size=20,
                   num_classes=19,
                   dim=1024,
                   depth=4,
                   heads=16,
                   mlp_dim=int(2048/2),
                   dropout=0.1,
                   emb_dropout=0.1,
                   channels=1).to("cuda:0")

model = init_model()
model.load_state_dict(torch.load(
    r"C:\Users\aashr\Desktop\research\testing_grounds\saved_models\ViT\WithNoise\best_model_state_dict_ViT_forau0io70q_stime_2023-03-27T06-39-26.729202__acc_0.9905545115470886__auc_0.9999259114265442.pt"))


In [ ]:
from dvrecorder import Recorder
rmodel = Recorder(model)
img = torch.randn(1, 1, 400, 400).to("cuda:0")
# model.eval()
preds,attns = rmodel(img)

preds,attns

In [ ]:
# from vit_pytorch.recorder import Recorder
# import torch
# from vit_pytorch.vit import ViT
# # from vit_pytorch.deepvit import DeepViT as ViT
# v = ViT(
#     image_size=256,
#     patch_size=32,
#     num_classes=1000,
#     dim=1024,
#     depth=6,
#     heads=16,
#     mlp_dim=2048,
#     dropout=0.1,
#     emb_dropout=0.1
# )

# # import Recorder and wrap the ViT
# v = Recorder(v)

# # forward pass now returns predictions and the attention maps

# img = torch.randn(1, 3, 256, 256)
# preds, attns = v(img)
# preds.shape,attns.shape

In [ ]:
from dvrecorder import Attention as DVAttention

In [ ]:
for i in model.modules():
    if i == DVAttention:
        [print(j,"\n") for j in i.modules()]
        print("\n\n\n")
    break

In [144]:
for i in model.modules():
    for j in i.modules():
        print (j)
    break

DeepViT(
  (to_patch_embedding): Sequential(
    (0): Rearrange('b c (h p1) (w p2) -> b (h w) (p1 p2 c)', p1=20, p2=20)
    (1): LayerNorm((400,), eps=1e-05, elementwise_affine=True)
    (2): Linear(in_features=400, out_features=1024, bias=True)
    (3): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  )
  (dropout): Dropout(p=0.1, inplace=False)
  (transformer): Transformer(
    (layers): ModuleList(
      (0): ModuleList(
        (0): Residual(
          (fn): PreNorm(
            (norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
            (fn): Attention(
              (to_qkv): Linear(in_features=1024, out_features=3072, bias=False)
              (dropout): Dropout(p=0.1, inplace=False)
              (reattn_norm): Sequential(
                (0): Rearrange('b h i j -> b i j h')
                (1): LayerNorm((16,), eps=1e-05, elementwise_affine=True)
                (2): Rearrange('b i j h -> b h i j')
              )
              (to_out): Sequential(
    